# Geodata in Julia

The geodata ecosystem in Julia has matured a lot, but is not in a fully stable state yet.

My geodata skills are pretty basic, but here is what I know...

My stack:
- [Raster.jl](https://github.com/rafaqz/Rasters.jl) for raster data (geotiff, Netcdf, ascii-grid, etc)
- [Shapefile.jl](https://github.com/JuliaGeo/Shapefile.jl) for, you guessed, shapefiles
- [ArchGDAL.jl](https://github.com/yeesian/ArchGDAL.jl) for interactions with the GDAL lib
- [Proj4.jl](https://github.com/JuliaGeo/Proj.jl) for map projections

## Geo Ecosystem

- https://juliageo.org/ -- biggest geo-group
- https://github.com/JuliaEarth -- for geostatistics
- https://ecojulia.org/ -- (spatial)ecology
- https://github.com/GenericMappingTools/GMT.jl (for Huw)

## Raster data

(a good tutorial https://github.com/xKDR/datascience-tutorials)

First download some data:
using Pkg; Pkg.instantiate()

In [ ]:
using Downloads # ships with Julia
using Rasters, ZipFile
mkpath("data")
# download if not already downloaded
!isfile("data/dhm200.zip") && Downloads.download("https://data.geo.admin.ch/ch.swisstopo.digitales-hoehenmodell_25/data.zip", "data/dhm200.zip")
# this extracts the file we want from the zip-file (yep, a bit complicated)
zip = ZipFile.Reader("data/dhm200.zip")
write("data/dhm200.asc", read(zip.files[1]))
close(zip)

ra = Raster("data/dhm200.asc")

### Plot raster

In [ ]:
using Plots
plotly()  # use the Plotly.jl backend, this allows zooming withing the Jupyter notebook
plot(ra, ticks=:native,   # thus Rasters.jl provides a plot-receipt and plotting is easy
     size=(1000,700),     # make it bigger
     max_res=2000)        # Rasters downsamples before plotting to make plotting faster.  Max number of gridpoints

### Index raster

Rasters have powerful (but also complicated) indexing capabilities.

See https://rafaqz.github.io/Rasters.jl/stable/

In [ ]:
ra[X(Near(600000)), Y(Near(250876))]     # shows where the x-y are

In [ ]:
ra[X(Near(600000)), Y(Near(250876))][1]  # index with the [1] to get the value out

In [ ]:
ra[X(500000..550000), Y(130000..150000)] # a range

### Other raster operations

resample, mosaic, crop...

See the [docs](https://rafaqz.github.io/Rasters.jl/stable/#Methods-that-change-the-reslolution-or-extent-of-an-object)

### Shapefiles

Shapefiles contain vector polygons (and such)

First, download and extract data about zip-code (PLZ) areas in Switzerland

In [ ]:
!isfile("data/plz.zip") && Downloads.download("https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_SHP_LV03.zip", "data/plz.zip")
zip = ZipFile.Reader("data/plz.zip")
for f in zip.files
    name = basename(f.name)
    if startswith(name, "PLZO_PLZ")
        write("data/$(name)", read(f))
    end
end
close(zip)

### Shapefiles

Read it and select Zermatt (3920)

In [ ]:
using Shapefile
tab = Shapefile.Table("data/PLZO_PLZ.shp")

zermatt = findfirst(tab.PLZ.==3920)
plot(tab.geometry[zermatt])

### Shapefiles & DataFrames

Shapefiles contain tables of attributes which can be handled with DataFrames, if so desired

In [27]:
using DataFrames
DataFrame(tab)

Row,geometry,UUID,OS_UUID,STATUS,INAEND,PLZ,ZUSZIFF
,Polygon,String,String,String,String,Int64,Int64
1,Polygon(1772 Points),{0072F991-E46D-447E-A3BE-75467DB57FFC},{281807DC-9C0B-4364-9A55-0E8956876194},real,nein,3920,0
2,Polygon(1809 Points),{C3D3316F-1DFE-468E-BFC5-5C2308743B4C},{F065D58C-3F88-46EF-9AA0-DA0A96766333},real,nein,3864,0
3,Polygon(1389 Points),{479E660B-A0A5-4297-AA66-FA62735EFDC6},{45243689-766B-4FFC-9A14-AF0D17AADE48},real,nein,1948,1
4,Polygon(1525 Points),{FDFBFFDF-11C2-4CC9-B903-EF17677388BF},{678407FD-30DD-4699-A2D7-FD3602AD9EF0},real,nein,7504,0
5,Polygon(1100 Points),{CB229C54-DF46-45A0-B75F-6E77240B4B03},{D4A72AA9-CF35-4F14-8AD4-03F2EDAC5BA1},real,nein,3984,2
6,Polygon(1690 Points),{5B28EF42-165A-4C06-AC5B-BFB4068A7988},{922C55FE-EF7F-46EA-A14B-98D2BDD00B5C},real,nein,7530,0
7,Polygon(1477 Points),{6262A301-67E7-4095-BB32-9A62097205DE},{EAD450C6-15D2-4243-9DA6-F46D40B74A83},real,nein,3818,0
8,Polygon(1536 Points),{DCC5A542-C9B9-45B6-BD94-546E5B347A8F},{F744CFF2-ADAC-471D-A054-FE349F787851},real,nein,7250,0
9,Polygon(1263 Points),{195005AC-2846-4DD3-A72C-32765AF4032A},{3C6766E9-20A4-47BE-8AB5-C3FF1FF53553},projektiert,nein,7132,0


### Crop and mask raster

Read it and select Zermatt (3920)

In [ ]:
ra_z = crop(ra; to = tab.geometry[zermatt])
mask_z = mask(ra_z, with = tab.geometry[zermatt])
plot(mask_z)

# Exercise

- Download the Swiss Glacier Inventory 2016 from https://www.glamos.ch/en/downloads#inventories/B56-03
- look up Gornergletscher
- plot it into the last plot we just did
- mask the elevation map with the Gornergletscher outline and calculate the mean elevation